<a href="https://colab.research.google.com/github/w86w86/econ8320_US-Census/blob/main/MAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Call the API with my variables for my research

> List of some states "46 SD", "47 TN","10 DE", "2 AK","5 AR","24 MD","31 NE"

CBSA:

"46 SD",
* "43620 Sioux Falls, SD",

"47 TN"
* "16860 Chattanooga, TN-GA",
* "28940 Knoxville, TN",
* "34980": "Nashville-Davidson--Murfreesboro--Franklin, TN",
* "28700": "Kingsport-Bristol-Bristol, TN-VA",
* "32820": "Memphis, TN-MS-AR",
* "17300": "Clarksville, TN-KY",
* "27740": "Johnson City, TN",
* "17420": "Cleveland, TN",

"10 DE"
* "20100 Dover, DE",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "41540": "Salisbury, MD-DE",
*
* 0 city for "2 AK"

"5 AR",
* "22900 Fort Smith, AR-OK",
* "30780 Little Rock-North Little Rock-Conway, AR",
* "22220 Fayetteville-Springdale-Rogers, AR-MO",
* "38220 Pine Bluff, AR",
* "32820 Memphis, TN-MS-AR",
*
"24 MD",
* "25180": "Hagerstown-Martinsburg, MD-WV",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "12580": "Baltimore-Columbia-Towson, MD",
* "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
* "15680": "California-Lexington Park, MD",
* "41540": "Salisbury, MD-DE",

##CONSTANTES

In [ ]:
# (10) IMPORT LIBRARY
import requests
import pandas as pd
import regex as re
import time
from datetime import datetime as dt
import json
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# (20) CONSTANTES

#Dict of all states
#all_states = requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json')
#states = json.loads(all_states.text)
#states_dict = states['variables']['STATE']['values']['item']
#states = [state for state in states_dict.keys()]

#All year of the study


###(10) CLASS API

In [ ]:
# (30) CLASS API and Decorator [log_activity2]

def log_activity2(func):
    def wrapper(*args, **kwargs):
        gdrivePath='/content/gdrive/MyDrive/project/'
        # Create a logger
        logger = logging.getLogger('activity_logger')
        logger.setLevel(logging.INFO)

        # Create a file handler and set the logging format
        file_handler = logging.FileHandler(gdrivePath + 'activity.log')
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # Add the file handler to the logger
        logger.addHandler(file_handler)

        # Log the activity before calling the wrapped function
        logger.info(f'Activity: {func.__name__}')

        # Call the wrapped function
        return func(*args, **kwargs)

    return wrapper


class API:
  """This generic class API will handle at once all request to the CPS api
  How to?:
  > instanciate a variable at once, then use it to browse through all variables (not data) on the Census API json file
  > example api:    api=API()

  Properties:
  > allVars_dict (variable): a dict of all variables for each 'year' (key)
  > examples:
    - api.allVars_dict[2023]        * to se all the variables (dict) in 2023
    - 'GTCBSA' in api.allVars_dict[2024].keys() *                is <GTCBSA> in 2024?
    - api.allState_dict[2021]       * all stateID:description in year 2021"""

  def __init__(self) -> None:
    self.get()

  @log_activity2
  def get(self):
    self.allVars_dict = {year_: year_vars for year_, year_vars in \
            zip(self.allYearList(), [json.loads(requests.get('https://api.census.gov/data/'+ str(year_) +'/cps/basic/jan/variables.json').text)['variables'] for year_ in self.allYearList()])}

  def store(self):
    pass

  @log_activity2
  def allStatesIdList(self, year:int=2020):
    return list(map(int, self.allVars_dict[year]['GESTFIPS']['values']['item'].keys())) #convert in int

  @log_activity2
  def allMonthList3TLA(self):
    month_dict = {1:"jan",2:"feb",3:"mar",4:"apr",5:"may",6:"jun",7:"jul",8:"aug",9: "sep",10: "oct",11:"nov",12:"dec"}
    #return next(key for key, value in month_dict.items() if value == m)
    return list(month_dict.values())

  def allYearList(self):
    return range(2010, 1+dt.now().year)

  @log_activity2
  def getVarsNameFromUrl(self, url):
    return re.findall(r'([A-Z]+)', url)


api=API()

INFO:activity_logger:Activity: get


###(20) CLASS FileCompMix:

In [ ]:
class FileCompMix:
  """This class will track the which State Year and Month has already been processed
  so we are ending up re-download the same file, knowing that we have close to 10,000 calls
  to make with an average of 2.50 secs = ~ 7 hours in total.

  Hou to?:
  > track = FileCompMix(filename [optional])
        - if filename contains any char then the class will generate a new fileName_%m%d-%H%M%.
        - defaut is   ' MixCompStatus.csv '
  > two main methods:
        - track.record_state_of_state_year_month()   *   to store the info
        - track.is_record_state_of_state_year_month  *   to check if elt exist already
  """
  def __init__(self, fileName='', gdrivePath='/content/gdrive/MyDrive/project/') -> None:
    if not len(fileName) == 0:
      fileName  = dt.now().strftime("_%m%d-%H%M%S")
    self.gdrivePath = gdrivePath
    self.track_file = self.gdrivePath + 'MixCompStatus' + fileName + ".csv"
    self.set_df()
    self.eltInFileList = [list(i) for i in self.df.values]


  def set_df(self):
    try:
      self.df = pd.read_csv(self.track_file, header=None)
    except Exception as e:
      self.df = pd.DataFrame([])
    self.hasCSVdata = self.df.shape[0]

  def numberOfelement(self):
    return self.df.shape[0]

  def record_state_of_state_year_month(self, element):
    self.df = pd.concat([self.df, pd.DataFrame([element])], ignore_index=True)
    #print (f'__record_state_of_state_year_month__ len: {self.df.shape[0]}')
    self.df.to_csv(self.track_file, mode='w', header=False, index=False) #not mode='a'
    self.hasCSVdata = True

  def is_record_state_of_state_year_month(self, element):
    if not self.hasCSVdata:
      #print ('--------------------------------#1F File emply, so will add')
      return False
    else:
      #print('______________________')
      #print(f'------- element: {element}')
      #print(f'------- self.eltInFileList: {self.eltInFileList}')
      #print('______________________')
      if (list(element) in self.eltInFileList):
        #print ('--------------------------------#2T elt already there, so not added, skipped')
        return True
      #print ('--------------------------------#3F, so will add')
      return False

###(40) CLASS MIXCOMP

In [ ]:
# (40) CLASS MIXCOMP
#Decorator to track API response time
def track_response_time(func):
    def wrapper(self, *args, **kwargs):
        start_time = time.time()
        result = func(self, *args, **kwargs)
        end_time = time.time()
        response_time = end_time - start_time
        self.response_time = response_time  # Store in the class
        print(f"Function: '{func.__name__}()'. Census API query took: {response_time} seconds")
        return result
    return wrapper


#Class for unit-request (specific State, specific Year, specific Month)
class MixComp:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
  def __init__(self, stateID, year, month):
      key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
      self.url = 'https://api.census.gov/data/'+ str(year) +'/cps/basic/'+ month + '?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:' + str(stateID) + key
      self.gdrivePath='/content/gdrive/MyDrive/project/'
      self.activityLog = self.gdrivePath + 'activity.log'
      self.stateID = stateID
      self.year = year
      self.month = month
      self.response_time = 0 # how long the API took to run on Census CPS
      self.dataAPI = None
      self.dataText = None
      self.data4Df = []
      self.df = pd.DataFrame(None) #Will contain the final df(dataframe)
      self.initLogger()

  def initLogger(self):
      # Create a logger
      self.logger = logging.getLogger('activity_logger')
      self.logger.setLevel(logging.INFO)
      # Create a file handler and set the logging format
      file_handler = logging.FileHandler(self.activityLog)
      formatter = logging.Formatter('%(asctime)s - %(levelname)s - [w86] %(message)s')
      file_handler.setFormatter(formatter)
      # Add the file handler to the logger
      self.logger.addHandler(file_handler)

  def __str__(self):
      return f"""
      Mix Composition of the US population: stateID:={self.stateID}, month={self.month}, year={self.year}
      Dataframe head:
         {self.df.head(3)}"""

  def __repr__(self):
      return f"MixComp({self.stateID}, {self.year}, {self.month})"

  def __len__(self):
      return self.df.shape[0]

  def __add__(self, other):
      return pd.concat([self.df, other.df], axis=0)

  # to convert in class
  def checkVariable(self, year, url):
      """
      this function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
      """
      api=API()
      api.get()
      matches = api.getVarsNameFromUrl(url)
      cps_vars = api.allVars_dict[year].keys()
      res = dict()
      for var in matches:
        res[var] = True if var in cps_vars else False
      res['status'] = all([status for status in res.values()])
      return res

  @track_response_time
  def get(self):

      #This section will validate all variables and replace the city's with the right one if year>=2024
#      check = self.checkVariable(self.year, self.url)
#      print (f"checkVariable() = {check}")
#      if not check['status']:
#        print(f"BEFORE - Variables = {check}")
#        if (self.year>=2024) and ('CBSA' not in check.keys()): #before 2024, City variable in 'GTCBSA', starting on 2024, City variable was 'CBSA'
#          self.url = re.sub('GTCBSA', 'CBSA', self.url)
#          print(f"AFTER -  Variables = {self.checkVariable(self.year, self.url)}\nNew URL: {self.url}")
#        else:
#          raise(f"Please check all your Census variables. Unable to find some in the API call. Variables = {check}")

      try:
        # download the Demographics_Native_Country_Of_Mother for Mix-Compostion
        self.dataAPI = requests.get(self.url)
        res = dict({'response time in secs': round(self.response_time,2), 'status': True if 200==self.dataAPI.status_code else False, 'code':self.dataAPI.status_code, 'comment': 'url:' + self.url})
        self.logger.info(f'> Return dict: [[[[ {res} ]]]]')
        return res

      except requests.exceptions.RequestException as e:
          # Handle connection errors or HTTP errors
          #raise("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")
          self.logger.info(f'> STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[ {e} ]]]]')
          self.logger.info(f'> Skipped !!')
      except AttributeError:
          self.logger.info(f'> STOP ERROR: Please check your variable name!')

  def cleaning(self):
      self.dataText = self.dataAPI.text
      pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)
      pattern_cleaning = r'\n' #remove retunr line
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)
      self.logger.info(f'> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)')

  def preping(self):
      ##Separe cols from data (numbers)
      pattern = r'.*?(\["tabulate".*?"GTCBSA"\]),([\s\S]*)'
      matches = re.search(pattern, self.dataText)
      data_cols = matches.group(1)
      data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.

      self.data_cols = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

      #Transform string_row in List of Lists then Matrix
      rows = re.findall(r'\[(.*?)\]', data_vals)
      for row in rows:
        row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
        self.data4Df.append([int(elt) for elt in row_list])

      self.logger.info(f'> len(self.data4Df): {len(self.data4Df)}')

  def buildingDf(self):
      """
      ##Definition of variables:
      * tabulate: represents the information related to population counts.
      * state: column provides information about the state
      1. PEMNTVTY Demographics-native country of mother
      2. PEMARITL Demographics-marital status
      3. PESEX Demographics-sex
      4. PRCITSHP Demographics-United States citizenship group
      5. GTCBSA Demographics-city level
      We will understand the population composition by state about how mixed it is based on the native country of mothers,
      marital status distribution, gender demographics, citizenship status, at the city-level characteristics.
      This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.
      """
      ##Convert the result to the Df **mix_composition**
      self.df = pd.DataFrame(self.data4Df, columns=self.data_cols)

      ##Building the YYYYMM column
      self.df['YYYYMM'] = pd.Series([dt.strptime(str(self.year)+ '-' +str(self.month), "%Y-%b") for _ in range(self.df.shape[0])])

      self.logger.info(f'> Built Dataframe for stateiD:{self.stateID}, YYYYMM:{str(self.year)}-{str(self.month)}')

  def save(self, file=''):
      try:
        if file == '':
          stateID = '0'+ str(self.stateID) if len(str(self.stateID))<2 else str(self.stateID) #fixing stateID in 2 digits
          file = str(self.year)+'-'+ str(self.month)+'-'+ str(stateID)+'-MixComp.csv'
        r = self.df.to_csv( self.gdrivePath + file, index=False)
        self.logger.info(f'> Created CSV file: {file}')
        return r
      except FileNotFoundError:
        raise (f'Check the path {file} of the file')
        self.logger.info(f'> Check the path {file} of the file')
        self.logger.info(f'> Skipped {file} creation')

  def build(self):
      self.logger.info(f"> {'v'*60}")
      self.logger.info(f'> Called main(self) method - Entrance')
      self.logger.info(f'> Input Variables = stateID:{self.stateID},year:{self.year},month:{self.month}, self.url: {self.url}')

      self.logger.info(f'> Called main(self) - get() method - Will take some time usually 3-5min')
      url_call = self.get()
      self.logger.info(f'Called main(self) - line: get() method - Result:{url_call}')

      try:
        if url_call['status']:
          self.cleaning()
          self.preping()
          self.buildingDf(); self.logger.info(f'> Called main(self) - buildingDf() method')

          self.logger.info(f'> Called main(self) - save() method')
          self.save()
        else:
          self.logger.info(f"> Error during URL call, Code return: {url_call['code']}, Skipped !!")
      except Exception as e:
        self.logger.info(f"> Error during URL call, Code return was undefinied, Skipped !!")
      self.logger.info(f'> Called main(self) method - End')
      self.logger.info(f"> {'^'*60}")


###(60) CORE

In [ ]:
# (60 - 2) Going through All Months at once
from itertools import product
import random

# Serialize (9180 in tot :( wow )
#state_year_month = list( map( list, product(
#                                api.allStatesIdList(),
#                                api.allYearList(),
#                                api.allMonthList3TLA()) ) )

state_year_month = list( product(
                                api.allStatesIdList(),
                                api.allYearList(),
                                api.allMonthList3TLA()) )

# Shuffle
random.shuffle(state_year_month)

#logging.disable(logging.CRITICAL)
logging.disable(logging.NOTSET)

track = FileCompMix()

count = {'total': len(state_year_month), 'initiallyInFile':track.numberOfelement(), 'processed': 0, 'skipped': 0}
print ('Begin>',count)

for elt in state_year_month:
  if not track.is_record_state_of_state_year_month(elt):
    dataA = MixComp(*elt) #(stateID, year, month)
    dataA.build()

    if count['processed']%500==0: print ('+', end='')
    count['processed'] += 1
    track.record_state_of_state_year_month(elt)
  else:
    count['skipped'] += 1
    if count['skipped']%1000==0: print ('.', end='')

print ('\nEnd  <',count)
print (f"count['processed'] + count['skipped'] ?= count['total'] => [{count['processed'] + count['skipped'] } ?= {count['total']}]")

INFO:activity_logger:Activity: allStatesIdList
INFO:activity_logger:Activity: allMonthList3TLA
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:27,year:2022,month:jul, self.url: https://api.census.gov/data/2022/cps/basic/jul?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:27&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(self) - get() method - Will take some time usually 3-5min


Begin> {'total': 9180, 'initiallyInFile': 81, 'processed': 0, 'skipped': 0}


INFO:activity_logger:> Return dict: [[[[ {'response time in secs': 0, 'status': True, 'code': 200, 'comment': 'url:https://api.census.gov/data/2022/cps/basic/jul?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:27&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48'} ]]]]
INFO:activity_logger:Called main(self) - line: get() method - Result:{'response time in secs': 0, 'status': True, 'code': 200, 'comment': 'url:https://api.census.gov/data/2022/cps/basic/jul?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:27&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48'}


Function: 'get()'. Census API query took: 162.20810317993164 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 136
INFO:activity_logger:> Built Dataframe for stateiD:27, YYYYMM:2022-jul
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Called main(self) - save() method
INFO:activity_logger:> Created CSV file: 2022-jul-27-MixComp.csv
INFO:activity_logger:> Called main(self) method - End
INFO:activity_logger:> ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:10,year:2011,month:may, self.url: https://api.census.gov/data/2011/cps/basic/may?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:10&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(s

...

INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 194
INFO:activity_logger:> Built Dataframe for stateiD:10, YYYYMM:2022-oct
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Called main(self) - save() method
INFO:activity_logger:> Created CSV file: 2022-oct-10-MixComp.csv
INFO:activity_logger:> Called main(self) method - End
INFO:activity_logger:> ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:27,year:2018,month:may, self.url: https://api.census.gov/data/2018/cps/basic/may?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:27&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(s

Function: 'get()'. Census API query took: 194.44279503822327 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 129
INFO:activity_logger:> Built Dataframe for stateiD:27, YYYYMM:2018-may
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Called main(self) - save() method
INFO:activity_logger:> Created CSV file: 2018-may-27-MixComp.csv
INFO:activity_logger:> Called main(self) method - End
INFO:activity_logger:> ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:51,year:2019,month:mar, self.url: https://api.census.gov/data/2019/cps/basic/mar?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:51&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(s

Function: 'get()'. Census API query took: 198.40798807144165 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 435
INFO:activity_logger:> Built Dataframe for stateiD:51, YYYYMM:2019-mar
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Called main(self) - save() method
INFO:activity_logger:> Created CSV file: 2019-mar-51-MixComp.csv
INFO:activity_logger:> Called main(self) method - End
INFO:activity_logger:> ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:45,year:2022,month:nov, self.url: https://api.census.gov/data/2022/cps/basic/nov?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:45&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(s

 _

###TEST section

In [ ]:
# (60) Going through All Months at once
#for year in api.allYearList():
#  for stateID in api.allStatesIdList(): #['31', '47']:
#    for month in api.allMonthList3TLA(): #['jan', 'feb']:
#      dataA = MixComp(stateID=stateID, year=year, month=month)
#      dataA.build()

why skipped?

2019-apr-45-MixComp (it is there, just in the as header :) in Colab)

Those two site failed - but fixed now with Exception

https://api.census.gov/data/2024/cps/basic/oct?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:17&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48


Need to 'Run all' to take changes
https://api.census.gov/data/2024/cps/basic/dec?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:33&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48



INFO:activity_logger:> vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = stateID:32,year:2014,month:jul, self.url: https://api.census.gov/data/2014/cps/basic/jul?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:32&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
INFO:activity_logger:> Called main(self) - get() method - Will take some time usually 3-5min
INFO:activity_logger:> STOP ERROR: Tools for Data Science - Semester Project: An error occurred:
 [[[[ ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)) ]]]]
INFO:activity_logger:> Skipped !!
INFO:activity_logger:Called main(self) - line: get() method - Result:None
Function: 'get()'. Census API query took: 200.60595393180847 seconds
- - -
<ipython-input-15-4270161bdba5> in build(self)
    182       self.logger.info(f'Called main(self) - line: get() method - Result:{url_call}')
    183
--> 184       if url_call['status']:
    185         self.cleaning()
    186         self.preping()

TypeError: 'NoneType' object is not subscriptable